In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 下载训练集
train_dataset = datasets.MNIST(root='./', train=True, transform=transforms.ToTensor(), download=True)
# 下载测试集
test_dataset = datasets.MNIST(root='./', train=False, transform=transforms.ToTensor(), download=True)

# 批次大小
batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# 装载测试集
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2, 2))
        self.fc1 = nn.Sequential(nn.Linear(64 * 7 * 7, 1000), nn.Dropout(p=0.4), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(1000, 10), nn.Softmax(dim=1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

LR = 0.0003
# 定义模型
model = Net()
# 定义代价函数
entropy_loss = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(model.parameters(), LR)

# 记录训练损失和准确率
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

def train():
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for data in train_loader:
        inputs, labels = data
        out = model(inputs)
        loss = entropy_loss(out, labels)

        # 梯度清0
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(out, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)

    return avg_loss, accuracy

def test():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            out = model(inputs)
            loss = entropy_loss(out, labels)
            running_loss += loss.item()

            _, predicted = torch.max(out, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(test_loader)
    accuracy = correct / total
    test_losses.append(avg_loss)
    test_accuracies.append(accuracy)

    return avg_loss, accuracy

# 训练与测试模型
for epoch in range(20):
    print('epoch:', epoch)
    train_loss, train_accuracy = train()
    test_loss, test_accuracy = test()
    print(f'Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}')
    print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

# 绘制loss曲线
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 绘制accuracy曲线
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()